# 1.9 Proyecto de Cierre (Nivel Junior)

**Resumen Ejecutivo**: Integra todo lo aprendido (SELECT, JOIN, filtros, agregaciones, subconsultas) en un proyecto real: dashboard comercial con KPIs de ventas, productos y clientes.

---

## 🎯 Objetivos de Aprendizaje
- Construir consultas complejas combinando JOINs, agregaciones y filtros.
- Generar métricas ejecutivas (ventas totales, top productos, ticket promedio).
- Detectar productos sin movimiento y anomalías en transacciones.

## 🧰 Prerrequisitos
- Haber completado `08_calidad_datos_basica.ipynb` y todos los módulos Level01.
- Dataset completo ejecutado (`dataset_setup.sql`).
- Capacidad de analizar enunciados de negocio y traducirlos a SQL.

---

## 📚 Enunciado del Proyecto

Eres analista junior. El equipo comercial necesita un mini dashboard con: 
1. Ventas totales y margen total.
2. Top 5 productos por unidades.
3. Clientes nuevos últimos 30 días.
4. Ticket promedio por región.
5. Productos sin ventas (catalogados pero nunca vendidos).
6. Ventas con anomalías (descuento >50% o cantidad <=0).
7. Participación de cada segmento (por unidades y por margen).

## Enunciado

Eres analista junior. El equipo comercial necesita un mini dashboard con: 
1. Ventas totales y margen total.
2. Top 5 productos por unidades.
3. Clientes nuevos últimos 30 días.
4. Ticket promedio por región.
5. Productos sin ventas (catalogados pero nunca vendidos).
6. Ventas con anomalías (descuento >50% o cantidad <=0).
7. Participación de cada segmento (por unidades y por margen).

In [ ]:
SELECT SUM(cantidad) AS unidades_totales,
       SUM(cantidad * precio_unitario * (1 - descuento_pct/100.0)) AS ingresos_est,
       SUM(cantidad * (precio_unitario - costo_unitario)) AS margen_total
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id;

In [ ]:
SELECT TOP 5 p.producto_id, p.nombre, SUM(fv.cantidad) AS unidades
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
GROUP BY p.producto_id, p.nombre
ORDER BY unidades DESC;

In [ ]:
SELECT cliente_id, nombre, fecha_alta
FROM dbo.dim_clientes
WHERE fecha_alta >= DATEADD(DAY,-30,CAST(GETDATE() AS DATE));

In [ ]:
SELECT r.nombre_region,
       AVG(fv.cantidad * p.precio_unitario * (1 - descuento_pct/100.0)) AS ticket_promedio
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
JOIN dbo.dim_clientes c ON fv.cliente_id = c.cliente_id
JOIN dbo.dim_regiones r ON c.region_id = r.region_id
GROUP BY r.nombre_region;

In [ ]:
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
LEFT JOIN dbo.fact_ventas fv ON p.producto_id = fv.producto_id
WHERE fv.producto_id IS NULL;

In [ ]:
SELECT venta_id, fecha, producto_id, cliente_id, cantidad, descuento_pct
FROM dbo.fact_ventas
WHERE descuento_pct > 50 OR cantidad <= 0;

In [ ]:
SELECT c.segmento,
       SUM(fv.cantidad) AS unidades,
       SUM(fv.cantidad * (p.precio_unitario - p.costo_unitario)) AS margen
FROM dbo.fact_ventas fv
JOIN dbo.dim_clientes c ON fv.cliente_id = c.cliente_id
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
GROUP BY c.segmento;

In [ ]:
-- Sugerencia de estructura de CTEs (no ejecutar si no se desea):
-- WITH ventas_globales AS ( ... ), top_productos AS ( ... ), clientes_nuevos AS ( ... ),
-- ticket_region AS ( ... ), sin_ventas AS ( ... ), anomalías AS ( ... ), segmento AS ( ... )
-- SELECT * FROM ventas_globales;
-- SELECT * FROM top_productos;
-- ...

---

## ⚠️ Errores Comunes
- No aislar filtros correctamente (fechas, segmentos).
- Duplicar productos en agregaciones (JOIN mal definido).
- Inconsistencia de fechas (usar GETDATE sin considerar timezone o conversión).


---

## 🧪 Lista de Tareas
- 🟢 Paso 1: Métricas globales (unidades, ingresos, margen).
- 🟢 Paso 2: Top 5 productos por unidades.
- 🟠 Paso 3: Clientes nuevos últimos 30 días.
- 🟠 Paso 4: Ticket promedio por región.
- 🟠 Paso 5: Productos sin ventas (catalogados pero nunca vendidos).
- 🔴 Paso 6: Ventas con anomalías (descuento >50% o cantidad <=0).
- 🔴 Paso 7: Participación por segmento (unidades y margen).
- ⚙️ **Desafío Senior (Opcional)**: Crear una sola consulta con CTEs que devuelva todas las tablas anteriores en salida separada usando múltiples CTE y SELECT finales etiquetados.

---

## ✅ Conclusiones
- Este proyecto integra todos los conceptos de Level01: SELECT, filtros, JOINs, agregaciones, subconsultas.
- Construir dashboards ejecutivos requiere dominio de granularidad, JOINs correctos y validación de calidad.
- La práctica iterativa consolida el conocimiento y prepara para proyectos reales.

---

## 💼 Perspectiva de Negocio
- Dashboards ejecutivos bien construidos aceleran decisiones estratégicas.
- Detectar anomalías proactivamente evita pérdidas operativas y financieras.
- Consolidar métricas en una sola fuente (SQL) garantiza consistencia entre equipos.

---

## 🔖 Pie Editorial

**Curso**: Fundamentos de SQL Server - Nivel 1  
**Módulo**: 1.9 Proyecto de Cierre (Nivel Junior)  
**Versión**: 2.0 (Actualizado Enero 2025)  
**Autor**: lraigosov / LuisRai  
**Licencia**: Uso educativo - Atribución requerida

> 💡 Nota sobre IA: Este material fue estructurado con asistencia de modelos de lenguaje (OpenAI GPT-4, Anthropic Claude); el contenido fue validado y curado por especialistas para evitar alucinaciones y asegurar aplicabilidad práctica.

---
## Navegación
[⬅️ Anterior](08_calidad_datos_basica.ipynb)
---
